In [1]:
import numpy as np
import pandas as pd
import os
from music21 import converter, instrument, note, chord, midi    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:


# Función para cargar y procesar archivos MIDI en secuencias de notas
def load_midi_files(data_path):
    notes = []
    for artist_folder in os.listdir(data_path):
        artist_path = os.path.join(data_path, artist_folder)
        for midi_file in os.listdir(artist_path):
            midi_path = os.path.join(artist_path, midi_file)
            try:
                # Intenta cargar el archivo MIDI
                midi_data = converter.parse(midi_path)
                parts = instrument.partitionByInstrument(midi_data)
                notes_in_song = []
                for part in parts.parts:
                    # Procesa solo las partes de piano
                    if 'Piano' in str(part):
                        notes_in_song += [str(note.pitch) for note in part.flat.notes]
                if notes_in_song:
                    notes.append(notes_in_song)
            except Exception as e:
                print(f"Error al cargar {midi_path}: {e}")
                continue
    return notes

# Cargar datos MIDI
data_path = "midi"
notes = load_midi_files(data_path)



C:\Users\zapat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\music21\stream\base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


Error al cargar midi\Air Supply\All Out Of Love.1.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Air Supply\All Out Of Love.2.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Air Supply\All Out Of Love.3.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Air Supply\All Out Of Love.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Air Supply\Come What May.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Air Supply\Even the Nights Are Better.1.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Air Supply\Even the Nights Are Better.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Air Supply\Goodbye.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Air Supply\Making Love Out of Nothing at All.1.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Air Supply\Making Love Out of Nothing at All.2.mid: 'Chord' object has no attribute 'pitch'
Error al c

C:\Users\zapat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=2, channel=None, data=b'Fl\x84che'>; getting generic Instrument
  warnings.warn(


Error al cargar midi\Al Stewart\On The Border.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Al Stewart\The Year of the Cat.1.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Al Stewart\The Year of the Cat.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Al Stewart\Time Passages.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Al Wilson\Show and Tell.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Alabama\Angels Among Us.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Alabama\Born Country.1.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Alabama\Born Country.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Alabama\Feels So Right.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Alabama\How Do You Fall in Love.mid: 'Chord' object has no attribute 'pitch'
Error al cargar midi\Alabama\I'm in a Hurry.1.mid: 'Chord' object has no attrib

In [6]:
print("h")

h


In [ ]:
# Preprocesamiento de notas
unique_notes = sorted(set(item for song in notes for item in song))
print("unique_notes: ", unique_notes)
note_to_int = {note: num for num, note in enumerate(unique_notes)}

sequence_length = 50
X = []
y = []
for song in notes:
    if len(song) > sequence_length:
        # Convertir la canción en secuencias de enteros
        int_song = [note_to_int[note] for note in song]
        for i in range(len(int_song) - sequence_length):
            X.append(int_song[i:i + sequence_length])
            y.append(int_song[i + sequence_length])

X = np.reshape(X, (len(X), sequence_length, 1))
X = X / float(len(unique_notes))  # Normalizar
y = to_categorical(y, num_classes=len(unique_notes))

# Crear el modelo RNN
model = Sequential([
    LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    LSTM(256, return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(256, activation='relu'),
    Dense(len(unique_notes), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo y guardar las métricas
history = model.fit(X, y, epochs=50, batch_size=64, validation_split=0.2)
model.save("music_model.keras")
# Graficar precisión y pérdida
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy over Epochs')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over Epochs')

plt.show()





In [ ]:
# Función para generar una melodía a partir del modelo
def generate_melody(model, start_sequence, melody_length=100):
    melody = start_sequence[:]
    for _ in range(melody_length):
        input_sequence = np.reshape(melody[-sequence_length:], (1, sequence_length, 1))
        input_sequence = input_sequence / float(len(unique_notes))
        predicted_note = model.predict(input_sequence, verbose=0)
        next_note = np.argmax(predicted_note)
        melody.append(next_note)
    return melody

# Generar melodía
start_sequence = X[0][:, 0]  # Usa la primera secuencia en el dataset
generated_sequence = generate_melody(model, start_sequence)

# Convertir la secuencia generada a MIDI
def create_midi_from_sequence(generated_sequence, output_filename="generated_melody.mid"):
    midi_stream = midi.MidiTrack()
    for note_idx in generated_sequence:
        note_value = unique_notes[note_idx]
        if '.' in note_value or note_value.isdigit():  # Es un acorde
            chord_notes = [note.Note(int(n)) for n in note_value.split('.')]
            chord_obj = chord.Chord(chord_notes)
            midi_stream.append(chord_obj)
        else:
            midi_stream.append(note.Note(note_value))
    midi_stream.write(output_filename)

# Guardar la melodía generada en un archivo MIDI
create_midi_from_sequence(generated_sequence)